In [112]:
import requests
import json

application_id = "xxxxxxxxxxxxxx"
user_id = "xxxxxxxxxxxxxx"
application_secret ="xxxxxxxxxxxxxx"


In [107]:
import getpass
user_password = getpass.getpass()

 ············


In [110]:
GROUP_ID = 'xxxxxxxxxxxxxx'  # The id of the workspace containing the report you want to embed
REPORT_ID = 'xxxxxxxxxxxxxx'  # The id of the report you want to embed


def get_access_token(application_id, application_secret, user_id, user_password):
    data = {
        'grant_type': 'password',
        'scope': 'openid',
        'resource': "https://analysis.windows.net/powerbi/api",
        'client_id': application_id,
        'client_secret': application_secret,
        'username': user_id,
        'password': user_password
    }
    token = requests.post("https://login.microsoftonline.com/common/oauth2/token", data=data)
    assert token.status_code == 200, "Fail to retrieve token: {}".format(token.text)
    #print("Got access token: ")
    #print(token.json())
    return token.json()['access_token']


In [114]:
access_token = get_access_token(application_id,application_secret,user_id,user_password)

## Create Dataset


In [115]:
data_str={
   "name":"SalesMarketing",
   "defaultMode": "Push",
   "tables":[
      {
         "name":"Sales",
         "columns":[
            {
               "name":"OrderNumber",
               "dataType":"string"
            },
            {
               "name":"OrderDate",
               "dataType":"DateTime"
            },
            {
               "name":"ShipDate",
               "dataType":"DateTime"
            },
            {
               "name":"ProductID",
               "dataType":"Int64"
            },
            {
               "name":"Quantity",
               "dataType":"Int64"
            },
            {
               "name":"UnitPrice",
               "dataType":"Double"
            },
            {
               "name":"DiscountAmount",
               "dataType":"Double"
            },
            {
               "name":"PromotionCode",
               "dataType":"string"
            },
            {
               "name":"OriginationStateID",
               "dataType":"Int64"
            },
            {
               "name":"TotalAmount",
               "dataType":"Double"
            }
         ]
      }
   ]
}

In [116]:
# This will create dataset in My Workspace
create_dataset_url ='https://api.powerbi.com/v1.0/myorg/datasets'
# To create on specific Workspace 
create_dataset_url_grp = f'https://api.PowerBI.com/v1.0/myorg/groups/{GROUP_ID}/datasets'
header = {'Authorization': f'Bearer {access_token}'}
create_dataset = requests.post(url=create_dataset_url_grp, headers=header, data=json.dumps(data_str))

In [117]:
create_dataset.status_code

201

In [118]:
create_dataset.json()

{'@odata.context': 'http://wabi-us-east2-redirect.analysis.windows.net/v1.0/myorg/groups/652c9b66-8898-4e5d-8b1c-810b8ed82194/$metadata#datasets/$entity',
 'id': '886823a1-4945-4a0e-addb-ea80087b5ba8',
 'name': 'SalesMarketing',
 'defaultRetentionPolicy': 'None',
 'targetStorageMode': 'Unknown',
 'upstreamDatasets': [],
 'schemaMayNotBeUpToDate': False,
 'users': []}

## Extract Dataset ID and table name

In [120]:
dataset_id=create_dataset.json()['id']
table_name=data_str['tables'][0]['name']

## Push row to dataset

In [121]:
content={
  "rows": [
    {
      "OrderNumber": "TT02260499",
      "OrderDate": "10/5/2021",
      "ShipDate": "10/6/2021",
      "ProductID": 12,
      "Quantity": 1,
      "UnitPrice": 324.95,
      "DiscountAmount": 0,
      "PromotionCode": "",
      "OriginationStateID": 3,
      "TotalAmount": 324.95
    },
    {
      "OrderNumber": "TT02260500",
      "OrderDate": "10/5/2021",
      "ShipDate": "10/6/2021",
      "ProductID": 12,
      "Quantity": 1,
      "UnitPrice": 324.95,
      "DiscountAmount": 0,
      "PromotionCode": "",
      "OriginationStateID": 3,
      "TotalAmount": 3899.4
    }
  ]
}

In [122]:
# This will post row to dataset on MyWorkspace
push_row_url = f'https://api.powerbi.com/v1.0/myorg/datasets/{dataset_id}/tables/{table_name}/rows'
# This will post row to dataset on Specific workspace
push_row_url_grp = f'https://api.powerbi.com/v1.0/myorg/groups/{GROUP_ID}/datasets/{dataset_id}/tables/{table_name}/rows'
header = {'Authorization': f'Bearer {access_token}'}
pushrow = requests.post(url=push_row_url_grp, headers=header, data=json.dumps(content))

In [123]:
pushrow.status_code

200